In [ ]:
import tensorflow as tf
import os
from pathlib import Path
# from tqdm import tqdm

In [ ]:
import pandas as pd
# import numpy as np

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Sequential, layers

In [ ]:
train_dir = Path("C:/Users/Testys/Documents/ML_Space New_Chapter/Computer Vision/Datasets/dogs-vs-cats/train")
test_dir = Path("C:/Users/Testys/Documents/ML_Space New_Chapter/Computer Vision/Datasets/dogs-vs-cats/test1")

In [ ]:
train_df = pd.DataFrame({'image_name':os.listdir(train_dir)})
train_df['label'] =train_df['image_name'].apply(lambda x: x.split('.')[0])
train_df

In [ ]:
test_df = pd.DataFrame({'image_name':os.listdir(test_dir)})
test_df['label'] =test_df['image_name'].apply(lambda x: x.split('.')[0])
test_df

In [ ]:
# dog_path_train = os.path.join(train_dir, 'dog')
# os.mkdir(dog_path_train)
# dog_df_train = train_df[train_df.label=='dog']


In [ ]:
# for n in tqdm(dog_df_train.image_name):
#     os.rename((os.path.join(train_dir, n)), (os.path.join(dog_path_train, n)))

In [ ]:
# cat_path_train = os.path.join(train_dir, 'cat')
# os.mkdir(cat_path_train)
# cat_df_train = train_df[train_df.label=='cat']


In [ ]:
# for n in tqdm(cat_df_train.image_name):
#     os.rename((os.path.join(train_dir, n)), (os.path.join(cat_path_train, n)))

In [ ]:
train_datagen = ImageDataGenerator(rescale= 1./255, validation_split= 0.3)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size= (150, 150),
    class_mode='binary',
    subset='training'
)

In [ ]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size= (150, 150),
    class_mode= 'binary',
    subset= 'validation'
)

In [ ]:
model = Sequential([
    layers.Conv2D(16, (3,3), activation= 'relu', input_shape =(150, 150, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation= 'relu'),
    layers.Dense(1,activation= 'sigmoid'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss= 'binary_crossentropy',
             optimizer = RMSprop(learning_rate= 0.001),
              metrics= ['accuracy']
             )

In [ ]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if logs.get('val_accuracy')>=0.90:
            self.model.stop_training = True
            
callback = mycallback()

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples//50,#batch_size, 
    epochs = 30,
    verbose=1,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples//50,#batch_size,
    callbacks=[callback]
)

350/350 [==============================] - 821s 2s/step - loss: 0.4793 - accuracy: 0.7706 - val_loss: 0.4391 - val_accuracy: 0.7946
Epoch 5/30
350/350 [==============================] - 829s 2s/step - loss: 0.4285 - accuracy: 0.8072 - val_loss: 0.4268 - val_accuracy: 0.8021
Epoch 6/30
350/350 [==============================] - 811s 2s/step - loss: 0.3829 - accuracy: 0.8292 - val_loss: 0.4107 - val_accuracy: 0.8135
Epoch 7/30
350/350 [==============================] - 776s 2s/step - loss: 0.3447 - accuracy: 0.8515 - val_loss: 0.3433 - val_accuracy: 0.8535
Epoch 8/30
350/350 [==============================] - 734s 2s/step - loss: 0.3112 - accuracy: 0.8631 - val_loss: 0.3390 - val_accuracy: 0.8581
Epoch 9/30
350/350 [==============================] - 926s 3s/step - loss: 0.2886 - accuracy: 0.8781 - val_loss: 0.3366 - val_accuracy: 0.8456
Epoch 10/30
350/350 [==============================] - 816s 2s/step - loss: 0.2698 - accuracy: 0.8850 - val_loss: 0.3131 - val_accuracy: 0.8621
Epoch 11/

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import os

#new = Path.('\dogs-vs-cats\test1')

images = os.listdir(r'C:/Users/Testys/Documents/ML_Space New_Chapter/Computer Vision/Datasets/dogs-vs-cats/test1')
print(images)



In [ ]:
for i in images:
    print()
    # predicting images
    path = r'C:/Users/Testys/Documents/ML_Space New_Chapter/Computer Vision/Datasets/dogs-vs-cats/test1/' + i
    # print(path)
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    results = []
    if classes[0]>0.5:
        results.append('Dog')
    if classes[0]< 0.5:
        results.append('Cat')

In [ ]:
print(results)

In [ ]:
img = os.listdir(r'C:\Users\Hp 15\Documents\My_files\Image Classification\tmp')
print(img)

In [ ]:
for i in images:
    print()
    # predicting images
    path = r'C:\Users\Hp 15\Documents\My_files\Image Classification\tmp/' + i
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    results = []
    if classes[0]>0.5:
        results.append('Dog')
    else:
        results.append('Cat')

In [ ]:
print(results)